<a href="https://colab.research.google.com/github/StereoSilence/Zoomcamp2025/blob/main/DEZoomcamp_dlt_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Workshop "Data Ingestion with dlt": Homework**


---

## **Dataset & API**

We’ll use **NYC Taxi data** via the same custom API from the workshop:

🔹 **Base API URL:**  
```
https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api
```
🔹 **Data format:** Paginated JSON (1,000 records per page).  
🔹 **API Pagination:** Stop when an empty page is returned.  

## **Question 1: dlt Version**

1. **Install dlt**:

In [1]:
 !pip install dlt[duckdb]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.3/884.3 kB 26.8 MB/s eta 0:00:00


> Or choose a different bracket—`bigquery`, `redshift`, etc.—if you prefer another primary destination. For this assignment, we’ll still do a quick test with DuckDB.

2. **Check** the version:


In [49]:
!dlt --version

dlt 1.6.1


or:

In [18]:
import dlt
print("dlt version:", dlt.__version__)

dlt version: 1.6.1


**Answer**:  
- Provide the **version** you see in the output.

## **Question 2: Define & Run the Pipeline (NYC Taxi API)**

Use dlt to extract all pages of data from the API.

Steps:

1️⃣ Use the `@dlt.resource` decorator to define the API source.

2️⃣ Implement automatic pagination using dlt's built-in REST client.

3️⃣ Load the extracted data into DuckDB for querying.



In [50]:
import dlt

BASE_API_URL = "https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api"

def paginated_getter():
    page_number = 1
    while True:
        params = {'page': page_number}
        try:
            response = requests.get(BASE_API_URL, params=params)
            response.raise_for_status()
            page_json = response.json()
            print(f'Got page {page_number} with {len(page_json)} records')

            if page_json:
                yield page_json
                page_number += 1
            else:
                break
        except Exception as e:
            print(e)
            break


pipeline = dlt.pipeline(destination="duckdb", dataset_name="nyc_taxi")
load_info = pipeline.run(nyc_taxi_data())
print(load_info)


Pipeline dlt_colab_kernel_launcher load step completed in 2.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset nyc_taxi
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1739207785.4134316 is LOADED and contains no failed jobs


Load the data into DuckDB to test:






In [51]:
load_info = pipeline.run(nyc_taxi_data)
# Проверяем, что именно загрузил dlt
print("🔍 Load info:", load_info)

🔍 Load info: Pipeline dlt_colab_kernel_launcher load step completed in 3.14 seconds
1 load package(s) were loaded to destination duckdb and into dataset nyc_taxi
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1739207827.5332923 is LOADED and contains no failed jobs


Start a connection to your database using native `duckdb` connection and look what tables were generated:

In [52]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,dlt_colab_kernel_launcher,nyc_taxi,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,dlt_colab_kernel_launcher,nyc_taxi,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,dlt_colab_kernel_launcher,nyc_taxi,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,dlt_colab_kernel_launcher,nyc_taxi,nyc_taxi_data,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


**Answer:**
* How many tables were created?

## **Question 3: Explore the loaded data**

Inspect the table `ride`:


In [53]:
df = pipeline.dataset(dataset_type="default").nyc_taxi_data.df()
df

,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1739205289.199574,zzBaE6Bsiknrrg,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1739205289.199574,+HPA6EAP+cGjcw,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1739205289.199574,ISqT1rJDhQbjDQ,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1739205289.199574,8JPrvA0tdBKL5w,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1739205289.199574,x5PT1s6YfMHw4A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,40.783522,-73.970690,5.7,1,CASH,40.778560,-73.953660,0.0,0.00,5.70,1.16,2009-06-19 11:28:00+00:00,2009-06-19 11:22:00+00:00,0.0,VTS,1739207827.5332923,K46V/aHbJtSdLw,NaN
99996,40.777200,-73.964197,4.1,1,CASH,40.779800,-73.974297,0.0,0.00,4.10,0.89,2009-06-17 07:43:00+00:00,2009-06-17 07:41:00+00:00,0.0,VTS,1739207827.5332923,uMCi6baFypDDxQ,NaN
99997,40.780172,-73.957617,6.1,1,CASH,40.788388,-73.976758,0.0,0.00,6.10,1.30,2009-06-19 11:46:00+00:00,2009-06-19 11:39:00+00:00,0.0,VTS,1739207827.5332923,qVXgHFK1ifbCRA,NaN
99998,40.777342,-73.957242,5.7,1,CASH,40.773828,-73.956690,0.0,0.00,6.20,0.97,2009-06-17 04:19:00+00:00,2009-06-17 04:13:00+00:00,0.5,VTS,1739207827.5332923,JvzUYvNkYla5IQ,NaN


**Answer:**
* What is the total number of records extracted?

## **Question 4: Trip Duration Analysis**

Run the SQL query below to:

* Calculate the average trip duration in minutes.

In [55]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM nyc_taxi_data;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]


**Answer:**
* What is the average trip duration?

## **Submitting the solutions**

* Form for submitting: TBA




## **Solution**

We will publish the solution here after deadline.